# XXX - Target group

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
from nltk import FreqDist, word_tokenize, corpus
import gensim
from gensim import corpora,models
from wordcloud import WordCloud
import pyLDAvis.gensim_models
from pprint import pprint

In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-df-2.csv', \
                 encoding = 'utf-8', header = 0)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-src-2.csv', \
                        encoding = 'utf-8', header =0)

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
len(src_tw_df.loc[src_tw_df.label==0])+len(df.loc[df.label==0]) #number of rumorous tweets

In [3]:
rumor_tw = df.cleaned_reply_tw[df.label ==0 ].append(src_tw_df.cleaned_src_tw[src_tw_df.label ==0])

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
non_rumor_tw = df.cleaned_reply_tw[df.label ==1 ].append(src_tw_df.cleaned_src_tw[src_tw_df.label ==1])

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
rumor_tw_list = rumor_tw.astype(str).apply(word_tokenize).tolist()
non_rumor_tw_list = non_rumor_tw.astype(str).apply(word_tokenize).tolist()

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Rumor LDA model

In [ ]:
# texts = rumor_tw_list+ non_rumor_tw_list

In [ ]:
#Rumor dictionary
id2word = corpora.Dictionary(rumor_tw_list)
print(id2word)

In [ ]:
# Term Document Frequency
corpus_r = [id2word.doc2bow(tw) for tw in rumor_tw_list]

In [ ]:
 # Build LDA model
lda_model_r = gensim.models.ldamodel.LdaModel(corpus=corpus_r,id2word=id2word,num_topics=5, random_state=100,
                                            update_every=1,chunksize=100,passes=10,alpha='auto',per_word_topics=True)

## Rumor topics 

In [ ]:
rumor_topics = pyLDAvis.gensim_models.prepare(lda_model_r,corpus_r,id2word)

In [ ]:
pyLDAvis.display(rumor_topics)

In [ ]:
p_r = pyLDAvis.gensim_models.prepare(lda_model_r, corpus_r, id2word)

In [ ]:
save_path = 'D:/論文/PHEME9/Code/graph/charliehebdo\\'
pyLDAvis.save_html(p_r, save_path+'lda_r.html')

In [ ]:
# Print the Keyword in the 5 topics
pprint(lda_model_r.print_topics())

# Non-rumor LDA model

In [ ]:
id2word = corpora.Dictionary(non_rumor_tw_list)
corpus_nr = [id2word.doc2bow(tw) for tw in non_rumor_tw_list]

In [ ]:
lda_model_nr = gensim.models.ldamodel.LdaModel(corpus=corpus_nr,id2word=id2word,num_topics=5, random_state=100,
                                            update_every=1,chunksize=100,passes=10,alpha='auto',per_word_topics=True)

## Non_rumor topics 

In [ ]:
non_rumor_topics = pyLDAvis.gensim_models.prepare(lda_model_nr,corpus_nr,id2word)

In [ ]:
pyLDAvis.display(non_rumor_topics)

In [ ]:
pprint(lda_model_nr.print_topics())

In [ ]:
p_nr = pyLDAvis.gensim_models.prepare(lda_model_nr, corpus_nr, id2word)

In [ ]:
save_path = 'D:/論文/PHEME9/Code/graph/charliehebdo\\'
pyLDAvis.save_html(p_nr, save_path+'lda_nr.html')

# Rumor TF-IDF

## gensim_models: TfidfModel

In [ ]:
model_tfidf_r = models.TfidfModel(corpus_r)

In [ ]:
tfidf_r = model_tfidf_r[corpus_r]
print(tfidf_r[0])

In [ ]:
td = {}
for document in tfidf_r:
    for token_id, score in document:
        current_score = td.get(id2word.get(token_id), 0)
        if current_score < score:
            td.update([(id2word.get(token_id), score)])

In [ ]:
# # Sort the items of ``td`` into a new variable ``sorted_td``
# # the ``reverse`` starts from highest to lowest
# sorted_td = sorted(td.items(), key=lambda kv: kv[1], reverse=True) 

# for term, weight in sorted_td[:50]: # Print the top 50 terms in the entire corpus
#     print(term, weight)

## sklearn package: TfidfVectorizer

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 100)
X = vectorizer.fit_transform(rumor_tw.astype(str).tolist())
terms_r = vectorizer.get_feature_names()

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
sums = X.sum(axis=0)

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
data = []
for col, term in enumerate(terms_r):
    data.append( (term, sums[0,col] ))

ranking = pd.DataFrame(data, columns=['term','rank']).sort_values('rank', ascending=False)
ranking[:50]

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,term,rank
52,not,1591.019064
8,charliehebdo,1003.982889
51,no,897.483263
67,say,799.099012
56,people,761.673205
32,kill,736.691114
13,dont,728.932808
46,muslims,666.081076
29,islam,665.576992
31,jesuischarlie,611.264677


# Non-rumor TF-IDF

In [37]:
vectorizer = TfidfVectorizer(max_features = 100)
Y = vectorizer.fit_transform(non_rumor_tw.astype(str).tolist())
terms_n = vectorizer.get_feature_names()

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\users\user\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [38]:
sums_n = Y.sum(axis=0)
d = []
for col, term in enumerate(terms_n):
    d.append( (term, sums_n[0,col] ))

ranking = pd.DataFrame(d, columns=['term','rank']).sort_values('rank', ascending=False)
ranking[:50]

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,term,rank
57,not,293.475942
37,kill,204.222638
56,no,202.762585
10,charliehebdo,201.820429
61,paris,199.959110
72,say,170.715009
3,attack,170.167696
63,people,157.148865
15,dont,153.455220
22,get,148.101476


# Common words (tf-idf)

In [40]:
vectorizer = TfidfVectorizer(max_features = 100)
Z = vectorizer.fit_transform(rumor_tw.append(non_rumor_tw).astype(str).tolist())
terms = vectorizer.get_feature_names()
sums = Z.sum(axis=0)


temp = []
for col, term in enumerate(terms):
    temp.append( (term, sums[0,col] ))

ranking = pd.DataFrame(temp, columns=['term','rank']).sort_values('rank', ascending=False)
ranking[:50]

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\users\user\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,term,rank
56,not,1902.278804
8,charliehebdo,1205.131161
55,no,1101.727721
69,say,967.795315
35,kill,949.348185
60,people,918.417452
16,dont,888.978372
32,islam,800.725530
49,muslims,791.856512
2,attack,770.050437
